<a href="https://colab.research.google.com/github/gabrielfernandorey/ITBA-NLP/blob/main/ITBA_nlp01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico NLP - Detección de Tópicos y clasificación
- ITBA 2024
- Alumno: Gabriel Rey
---

### Resumen del problema

- Calcular los tópicos de portales de noticias que se reciben 
- Frecuencia del cálculo de tópicos: diaria
- Colección de noticias: diariamente, en lotes o de a un texto.
- Identificar tópicos, entidades, keywords y análisis de sentimiento.

### Datos
- Se reciben las noticias con formato: Titulo, Texto, Fecha, Entidades, Keywords

### Tareas
- Modelo de detección de tópicos diario utilizando embeddings
- Definir un criterio de agrupación de tópicos aplicado al mismo día y entre distintos días (merging)
- Almacenar los embeddings de tópicos en una base de datos vectorial
- Modelo de datos dado: 
    - Id del tópico
    - Nombre del tópico
    - Keywords
    - Embbeding
    - Fecha de creación
    - Fecha de entrenamiento inicial
    - Fecha de entrenamiento actualizada
    - Umbral de detección
    - Documento mas cercano
---
Tareas en esta notebook:
- Inicializar la base de datos vectorial
- Ingestar data
- NER: Encontrar las entidades de cada documento
- Limpiar data
- Modelo: Armado del modelo BERTopic
- Entrenamiento
- Almacenamiento en base de datos vectorial


In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import json
from datetime import datetime
from dotenv import load_dotenv
from tqdm import tqdm
from collections import Counter

import spacy

from NLP_tools import clean_all
from core.functions import *

# -->> levantar la base antes de ejecutar
from opensearch_data_model import os_client
from opensearch_io import init_opensearch

### Inicializamos la base vectorial
Se modifica el indice de la base "Topic" agregando referencias del documento mas cercano como el ID y el titulo

In [2]:
# Inicialización de indices
init_opensearch()

El índice Topic ya existe. Saltando inicialización de base de datos.
El índice News ya existe. Saltando inicialización de base de datos.


### Path

In [3]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

'C:/Users/gabri/OneDrive/Machine Learning/Github/ITBA-NLP/data/'

### Data

In [ ]:
# Read the parquet file 

file = "train-00000-of-00001.parquet"
df_parquet = pd.read_parquet(PATH+file)

data = list(df_parquet['text'])

df_parquet.head(1)

In [5]:
# Cantidad total de documentos
len(data)

1000

In [17]:
df_parquet.sort_values("start_time_local", ascending=True, inplace=True)
df_out = df_parquet[df_parquet['start_time_local'].dt.date > pd.to_datetime('2024-05-31').date()]

In [18]:
df_out

,asset_id,title_ch,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
12498,110476946,Los Tipitos vuelven a Mendoza para celebrar 20...,Diario Los Andes,112583,2024-06-01 03:00:00,2024-06-01 00:00:00,"[Abel Pintos, Ricky Martin, Paulina Rubio]","[Ricky Martin, Peteco Carabajal, Chaqueño Pala...",2024-06-02 16:01:42.062757,"[{'entities': [{'end': 11, 'entity_group': 'OR...","[Tipitos, Mendoza, Los Tipitos, Plaza, Entrada...",Los Tipitos vuelven a Mendoza para celebrar 20...,Los Tipitos son una banda que marcaron su esti...,"[séptimo disco, camaleón, canción, banda, músi...",2024-06-02 16:10:29.363180,Los Tipitos son una banda que marcaron su esti...,Los Tipitos vuelven a Mendoza para celebrar 20...,0.146472,37.174823
6941,110467879,El ``RIGI`` afecta al patrimonio nacional y a ...,Diario Los Andes,11258,2024-06-01 03:00:00,2024-06-01 00:00:00,[],"[Cámara de Comercio Internacional, Poder Ejecu...",2024-06-02 12:21:38.590492,"[{'entities': [{'end': 8, 'entity_group': 'MIS...","[Na, ción, Estado Nacional, Gran Bretaña, Gell...","El ""RIGI"" afecta al patrimonio nacional y a nu...",El capítulo de la ley Bases: “Régimen de incen...,"[régimen, inversiones, patrimonio nacional, tr...",2024-06-02 12:36:30.679520,El capítulo de la ley Bases: “Régimen de incen...,"El ""RIGI"" afecta al patrimonio nacional y a nu...",0.247803,33.608522
9820,110468621,El jugador de Talleres que anotó un golazo par...,Vía País,3365,2024-06-01 03:19:12,2024-06-01 00:19:12,[La Voz],"[Colón, Brigadier López, Catriel Sánchez, Iván...",2024-06-02 12:27:59.973850,"[{'entities': [{'end': 22, 'entity_group': 'OR...","[Talleres, Colón, Colón de Santa Fe, Newell’s,...",El jugador de Talleres que anotó un golazo par...,Talleres se cruzará con Colón de Santa Fe el j...,"[colón, talleres, golazo, préstamo, almirante,...",2024-06-02 12:39:40.038500,Talleres se cruzará con Colón de Santa Fe el j...,El jugador de Talleres que anotó un golazo par...,0.194462,33.341122
4462,110468519,Cortes de luz programados para este sábado en ...,Diario El Litoral,3117,2024-06-01 04:07:14,2024-06-01 01:07:14,[],[],2024-06-02 12:28:55.717827,"[{'entities': [{'end': 54, 'entity_group': 'LO...","[Santa Fe, Carcaraña]",Cortes de luz programados para este sábado en ...,En el sur santafesino Preocupación por el robo...,"[cortes, cables, robo, preocupación, responsab...",2024-06-02 12:44:14.984180,En el sur santafesino Preocupación por el robo...,Cortes de luz programados para este sábado en ...,0.255352,32.616940
1781,110473152,Descifrando las calles: un viaje de 20 años po...,Diario El Litoral,3260,2024-06-01 04:25:15,2024-06-01 01:25:15,[],[],2024-06-02 14:34:01.744379,"[{'entities': [{'end': 80, 'entity_group': 'LO...","[Venado Tuerto, Vía y Obras Ven, ado Tuerto, C...",Descifrando las calles: un viaje de 20 años po...,En el edificio de Vía y Obras Venado Tuerto: d...,"[nomenclatura, tuerto, viaje, calles, edificio...",2024-06-02 14:47:40.586200,En el edificio de Vía y Obras Venado Tuerto: d...,Descifrando las calles: un viaje de 20 años po...,0.227456,34.373774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10047,110506945,Los Pumas seven lograron un brillante triunfo ...,Cba24n,88,2024-06-02 23:40:11,2024-06-02 20:40:11,[],"[Lucho González, Wade, Schulz, Tobías Wade, Gó...",2024-06-02 23:40:38.844764,"[{'entities': [{'end': 9, 'entity_group': 'ORG...","[Pumas, Francia, Madrid, Australia, Gómez Cora...",Los Pumas seven lograron un brillante triunfo ...,------------------ publicidad ----------------...,"[pumas seven, corrida, posesión argentina, par...",2024-06-02 23:48:18.920530,------------------ publicidad ----------------...,Los Pumas seven lograron un brillante triunfo ...,0.127799,0.135533
10805,110506987,Reapareció Posse en una fiesta de la Embajad

In [12]:
# Lote para pruebas
chunk = 200
data = data[:chunk]

### StopWords
Se genera una lista especial de stopwords

In [7]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv(PATH+'spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv(PATH+'spanish_stop_words_spec.csv' )['stopwords'].values)

In [ ]:
""" import csv
# Guardar la lista de stopwords especial en un archivo CSV
with open(PATH+"spanish_stop_words_spec.csv", mode='w', newline='', encoding='utf-8') as archivo:
    escritor = csv.writer(archivo)
    escritor.writerow(['stopwords'])
    for stopword in SPANISH_STOPWORDS_SPECIAL:
        escritor.writerow([stopword]) """

### NER - Named Entity Recognition
Obtener entidades de las noticias 

-   Nota: Aunque el dataset original provee keywords y entities, se realiza el proceso de obtención de los mismos y se utilizan para el modelo. 

In [8]:
# Cargar el modelo de spaCy para español
spa = spacy.load("es_core_news_lg")

In [ ]:
 
# Cargar o saltar carga y procesar celda inferior
with open(PATH+f'modelos/entities_{str(chunk)}.json', 'r') as json_file:
    entities = json.load(json_file)

with open(PATH+f'modelos/keywords_{str(chunk)}.json', 'r') as json_file:
    keywords = json.load(json_file)

with open(PATH+f'modelos/vocabulary_{str(chunk)}.json', 'r') as json_file:
    vocab = json.load(json_file) 

In [9]:
# Detectar entidades para todos los documentos usando spaCy
# se procesa utilizando un criterio de seleccion

entities = []
for data_in in tqdm(data):

    # Contabilizar palabras en doc original
    normalized_text = re.sub(r'\W+', ' ', data_in.lower())
    words_txt_without_stopwords = [word for word in normalized_text.split() if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL]
    words_txt_counter = Counter(words_txt_without_stopwords)
    words_counter = {elemento: cuenta for elemento, cuenta in sorted(words_txt_counter.items(), key=lambda item:item[1], reverse=True) if cuenta > 1}

    # Extraer entidades del doc segun atributos
    extract = spa(data_in)
    entidades_spacy = [(ent.text, ent.label_) for ent in extract.ents]
    ent_select = [ent for ent in entidades_spacy if ent[1] == 'PER' or ent[1] == 'ORG' or ent[1] == 'LOC' ]

    # Extraer entidades de "maximo 3 palabras"
    ent_max_3 = [ent[0] for ent in ent_select if len(ent[0].split()) <= 3]
    ent_clean = clean_all(ent_max_3, accents=False)
    ent_unique = list(set([ word for word in ent_clean if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL] ))

    ents_proc = {}
    for ent in ent_unique:
        
        # Criterio de selección 
        weight = 0
        for word in ent.split():
            if word in words_counter:
                weight += 1 /len(ent.split()) * words_counter[word]
        
        ents_proc[ent] = round(weight,4)

    ents_proc_sorted = {k: v for k, v in sorted(ents_proc.items(), key=lambda item: item[1], reverse=True) if v > 0}

    # Crear la lista preliminar de entidades procesadas por noticia 
    pre_entities = [key for key, _ in ents_proc_sorted.items()] 

    # Obtener las últimas palabras de cada entidad que tenga mas de una palabra por entidad
    last_words = list(set([ent.split()[-1] for ent in pre_entities if len(ent.split()) > 1 ]))

    # Eliminar palabra única si la encuentra al final de una compuesta
    pre_entities_without_last_word_equal = []
    for idx, ent in enumerate(pre_entities):
        if not (len(ent.split()) == 1 and ent in last_words):
            pre_entities_without_last_word_equal.append(ent)

    # Obtener las palabras únicas
    unique_words = [ ent.split()[0] for ent in pre_entities_without_last_word_equal if len(ent.split()) > 1 ]

    # Eliminar palabra única si la encuentra al comienzo de una compuesta
    pre_entities_without_first_word_equal = []
    for idx, ent in enumerate(pre_entities_without_last_word_equal):
        if not (len(ent.split()) == 1 and ent in unique_words):
            pre_entities_without_first_word_equal.append(ent)

    # obtener entidades filtradas
    if len(pre_entities_without_first_word_equal) > 10:
        umbral = 10 + (len(pre_entities_without_first_word_equal)-10) // 2
        filter_entities = pre_entities_without_first_word_equal[:umbral] 
    else:
        filter_entities = pre_entities_without_first_word_equal[:10]

    pre_original_entities = []
    # capturar las entidades en formato original
    for ent in filter_entities:
        pre_original_entities.append([elemento for elemento in ent_max_3 if elemento.lower() == ent.lower()])

    sort_original_entities = sorted(pre_original_entities, key=len, reverse=True)
    
    try:
        entities.append( [ent[0] for ent in sort_original_entities if ent] ) 
    except Exception as e:
        entities.append([])


100%|██████████| 200/200 [00:28<00:00,  7.11it/s]


In [10]:
len(entities)

200

In [13]:
# Grabar
with open(PATH+f'modelos/entities_{str(chunk)}.json', 'w') as file:
    json.dump(entities, file)

## Keywords
Obtener palabras clave de las noticias

In [14]:
# Detectar keywords para todos los documentos usando spaCy

keywords_spa = []
for doc in tqdm(data):
    extract = spa(doc)
    keywords_spa.append([(ext.text, ext.pos_) for ext in extract])  

100%|██████████| 200/200 [00:24<00:00,  8.19it/s]


### Keyboards with neighboards
- Se seleccionan keywords unigrama y bigrama mediante la funcion keywords_with_neighboards(), que a su vez llama a las funciones get_bigrams() y get_neighbor_words()

In [15]:
# Funcion para obtener keywords con combinaciones de bigramas
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    for word_index in range(adj_length_of_word_list):
        
        # Indexar la lista 
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        # Agregar a la lista de "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [16]:
# devolver las palabras más frecuentes que aparecen junto a una palabra clave en particular
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        # Extrae solo las palabras en minúsculas (no las etiquetas) para cada bigrama
        words = [word.lower() for word, label in bigram]        
        
        # Comprueba si la palabra clave está en el bigram
        if keyword in words:
            idx = words.index(keyword)
            for word, label in bigram:
                
                #Ahora nos centramos en la palabra vecina, no en la palabra clave
                if word.lower() != keyword:
                    #Si la palabra vecina coincide con la pos_label correcta, agregarla a la lista maestra
                    if label == pos_label or pos_label == None:
                        if idx == 0:
                            neighbor_words.append(" ".join([keyword, word.lower()]))
                        else:
                            neighbor_words.append(" ".join([word.lower(), keyword]))
                    
    return Counter(neighbor_words).most_common()

In [17]:
def keywords_with_neighboards(keywords_spa, POS_1='NOUN', POS_2='ADJ'):
    """
    Funcion que devuelve dos listas:
    - lista de keywords with neighboards (segun argumentos POS_1 y POS_2)
    - lista de keywords mas frecuentes (segun argumentos POS_1 y POS_2)
    """

    doc_kwn = []
    commons = []
    for keywords in keywords_spa:
    
        # Obtenemos las keywords del tipo (Universal Dependences) mas frecuentes de cada doc (spaCy format)
        words = []
        for k_spa in keywords:
            if k_spa[1] == POS_1:
                words.append(k_spa[0])

        cont_words = Counter(words)

        common = cont_words.most_common()
        commons.append( [com for com in common if com[1] > 1] )

        # Calcular un umbral de corte (en repeticiones) para los keywords obtenidos
            ## suma de todos los valores
        valores = [valor for _, valor in common]

            ## Calcular los pesos como proporcionales a los valores mismos
        pesos = np.array(valores) / np.sum(valores)

            ## Calcular el umbral ponderado, valor 2 o superior ( debe repetirse la keyword al menos una vez )
        threshold = max(2, round(np.sum(np.array(valores) * pesos),4))


        # Obtenemos los bigramas del doc        
        tokens_and_labels = [(token[0], token[1]) for token in keywords if token[0].isalpha()]

        bigrams = get_bigrams(tokens_and_labels)

        keywords_neighbor = []
        for item_common in common:
            if item_common[1] >= threshold or len(keywords_neighbor) < 6: # corte por umbral o menor a 6
                
                kwn = get_neighbor_words(item_common[0], bigrams, pos_label=POS_2)
                if kwn != []:
                    keywords_neighbor.append( kwn )

        sorted_keywords_neighbor = sorted([item for sublist in keywords_neighbor for item in sublist ], key=lambda x: x[1], reverse=True)
        
        doc_kwn.append(sorted_keywords_neighbor)

    return doc_kwn, commons

In [18]:
# obtenemos keywords with neighboards y keywords mas frecuentes
k_w_n, keyword_single = keywords_with_neighboards(keywords_spa)

In [19]:
# muestra
k_w_n[4]

[('generoso gesto', 1),
 ('galante actitud', 1),
 ('excéntricos años', 1),
 ('mesa completa', 1),
 ('ascendente carrera', 1),
 ('famoso restaurante', 1)]

In [20]:
# filtramos las que al menos se repiten una vez
filtered_k_w_n = [ [tupla[0] for tupla in sublista if tupla[1] > 1] for sublista in k_w_n ]

In [21]:
# muestra
filtered_k_w_n[1]

['ahorro interno',
 'sistema bancario',
 'descomunal tasa',
 'impuestos confiscatorios']

In [22]:
# Analizamos los keywords unigrama
keyword_single[1]

[('ahorro', 12),
 ('dólares', 8),
 ('tasa', 7),
 ('interés', 6),
 ('argentino', 4),
 ('ahorros', 4),
 ('impuestos', 4),
 ('sistema', 4),
 ('dólar', 3),
 ('pesos', 3),
 ('otra', 3),
 ('cosa', 3),
 ('ingreso', 3),
 ('consumo', 3),
 ('futuro', 3),
 ('capacidad', 3),
 ('crecimiento', 3),
 ('rentabilidad', 3),
 ('actividad', 3),
 ('exterior', 3),
 ('economía', 3),
 ('países', 3),
 ('moneda', 2),
 ('caso', 2),
 ('peso', 2),
 ('tiempo', 2),
 ('aumento', 2),
 ('tipo', 2),
 ('cambio', 2),
 ('capitales', 2),
 ('inversión', 2),
 ('población', 2),
 ('margen', 2),
 ('argentinos', 2),
 ('devaluación', 2),
 ('plazo', 2),
 ('días', 2),
 ('mensual-', 2),
 ('renta', 2),
 ('forma', 2),
 ('millones', 2),
 ('familia', 2),
 ('pago', 2),
 ('expropiaciones', 2)]

In [23]:
# Si un keyword unigrama coincide en los bigramas elegidos se descarta
# la cantidad de keywords se obtiene utilizando la media como umbral de corte

# Umbral
values = [value for sublist in keyword_single for _, value in sublist]
threshold = np.mean(values)

for i, sublist in enumerate(keyword_single):
    lista_k_w_n = list(set([word for sentence in filtered_k_w_n[i] for word in sentence.split()]))
    for tupla in sublist:
        if tupla[1] >= threshold and tupla[0] not in lista_k_w_n:
            filtered_k_w_n[i].append(tupla[0])

keywords = filtered_k_w_n      

In [24]:
keywords[1]

['ahorro interno',
 'sistema bancario',
 'descomunal tasa',
 'impuestos confiscatorios',
 'dólares',
 'interés',
 'argentino',
 'ahorros',
 'dólar',
 'pesos',
 'otra',
 'cosa',
 'ingreso',
 'consumo',
 'futuro',
 'capacidad',
 'crecimiento',
 'rentabilidad',
 'actividad',
 'exterior',
 'economía',
 'países']

In [25]:
# Grabar
with open(PATH+f'modelos/keywords_{chunk}.json', 'w') as file:
    json.dump(keywords, file)

#### BOW - Armado del vocabulario con las entidades y keywords

In [26]:
# Unificar Entities + Keywords + Keywords with neighboards
vocab = list(set().union(*entities, *keywords))
len(vocab)

2681

In [27]:
# Guardar vocabulario
with open(PATH+f'modelos/vocabulary_{chunk}.json', 'w') as file:
    json.dump(vocab, file)

### Preprocesar las noticias
Se realiza un preprocesamiento mínimo del texto, pero no se le quita el sentido semántico para que mediante SentenceTransformer se puedan capturar embeddings de mejor calidad.

In [28]:
clean_data = Cleaning_text()

proc_data = []
for data_in in tqdm(data):
    aux = clean_data.unicode(data_in)
    aux = clean_data.urls(aux)
    aux = clean_data.simbols(aux)
    aux = clean_data.escape_sequence(aux)
    aux = " ".join([ word for word in aux.split() if word.lower() not in SPANISH_STOPWORDS_SPECIAL])
    proc_data.append(aux)


100%|██████████| 200/200 [00:00<00:00, 786.02it/s]


In [29]:
# Grabar
with open(PATH+f'modelos/proc_data_{chunk}.json', 'w') as file:
    json.dump(proc_data, file)

In [ ]:
# muestra
proc_data[60]

### Guardar noticias en el indice news de la base

In [ ]:
# configurar  batch_size = ( ej.: 5000 ) si se supera el limite 100MB en elasticsearch por operacion
index_name = 'news'
bulk_data = []

for idx, text_news in tqdm(enumerate(data)):
    doc = {
        'index': {
            '_index': index_name,
            '_id': int(df_parquet.index[idx])
        }
    }
    reg = {
        'title': str(df_parquet.iloc[idx].title),
        'news' : str(text_news), 
        'author': str(df_parquet.iloc[idx]['media']),
        'vector': None,
        'keywords' : keywords[idx],
        'entities' : entities[idx],
        'created_at': parse(str(df_parquet.iloc[idx]['start_time_local'])).isoformat(),
        'process': False
    }
    bulk_data.append(json.dumps(doc))
    bulk_data.append(json.dumps(reg))

# Convertir la lista en un solo string separado por saltos de línea
bulk_request_body = '\n'.join(bulk_data) + '\n'

# Enviar la solicitud bulk
response = os_client.bulk(body=bulk_request_body)

if response['errors']:
    print("Errores encontrados al insertar los documentos")
else:
    print("Documentos insertados correctamente")


#### Funciones de pruebas

In [ ]:
# Encontrar la posicion en el df segun su ID
df_parquet.index.get_loc(105640350)

In [ ]:
def funcion_aux(ID):
    keywords_df = df_parquet[df_parquet.index==ID]['Keyword Name'].values[0]
    entities_df = df_parquet[df_parquet.index==ID]['Entity Name'].values[0]
    fila = df_parquet.index.get_loc(ID)
    print(f"Noticia ID: {ID} {df_parquet[df_parquet.index==ID]['in__title'].values}\n")
    print(f"Entities de dataframe: {entities_df}")
    print(f"Keywords de dataframe: {keywords_df}")
    print("-"*80)
    print(f"Fila: {fila}")
    print(f"Entities calculadas: {entities[fila]}")
    print(f"Keywords calculadas: {filtered_k_w_n[fila]}")

funcion_aux(105638862)

In [19]:
df_out.to_parquet('2024-06-01.parquet')